In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from datetime import date, datetime

from sklearn.metrics import confusion_matrix
from sklearn.metrics import normalized_mutual_info_score, mutual_info_score
from sklearn.feature_selection import mutual_info_regression

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression

from sklearn.pipeline import Pipeline

from matplotlib import colors
import seaborn as sns


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers


In [3]:
import seaborn as sns
sns.set_style("whitegrid")

In [4]:
noun_sentence_df_final_joined_df_selected_columns_use_min = pd.\
    read_feather('noun_sentence_df_final_joined_df_selected_columns_use_min_test_3mins.fea')

In [5]:
noun_sentence_df_final_joined_df_selected_columns_use_min = noun_sentence_df_final_joined_df_selected_columns_use_min.dropna()

In [6]:
noun_sentence_df_final_joined_df_selected_columns_use_min.columns

Index(['level_0', 'listingid', 'index_x', 'AVM_Error_List_x',
       'Listing_Error_List_x', 'Transdate_x', 'Transprice_x', 'total_value_x',
       'cj_living_area_x', 'effective_year_built_x',
       ...
       'sentence_composition_0', 'sentence_composition_1',
       'sentence_composition_2', 'sentence_composition_3',
       'sentence_composition_4', 'sentence_composition_5',
       'sentence_composition_6', 'sentence_composition_7',
       'sentence_composition_8', 'transaction_year'],
      dtype='object', length=158)

In [7]:
noun_sentence_df_final_joined_df_selected_columns_use = noun_sentence_df_final_joined_df_selected_columns_use_min

In [8]:
predictor_columns = [   'sentence_composition_7_y',
 'AVM_Error_Indicators_5_y',
 'sentence_composition_7',
 'AVM_Error_Indicators_4_y',
 'AVM_Error_Indicators_5_x',
 'AVM_Error_Indicators_0',
 'AVM_Error_Indicators_10_x',
 'AVM_Error_Indicators_11_y',
 'AVM_Error_Indicators_4_x',
 'cj_living_area_x',
 'AVM_Error_Indicators_6_x',
 'AVM_Error_Indicators_0_x',
 'avmValue_x']

In [9]:
predictor_columns_ann = [  'AVM_Error_Indicators_4_y',
 'sentence_composition_0_x',
 'AVM_Error_Indicators_0',
 'Listing_Error_Indicators_1_x',
 'AVM_Error_Indicators_4',
 'AVM_Error_Indicators_scale_x',
 'AVM_Error_Indicators_11_x',
 'AVM_Error_Indicators_11_y',
 'sentence_composition_7_y',
 'AVM_Error_Indicators_0_y',
 'sentence_composition_7',
 'sentence_composition_7_x',
 'Listing_Error_Indicators_4_x',
 'AVM_Error_Indicators_4_x',
 'AVM_Error_Indicators_8_x',
 'AVM_Error_Indicators_10_y',
 'AVM_Error_Indicators_7_x',
 'AVM_Error_Indicators_10_x',
 'AVM_Error_Indicators_1_x',
 'cj_living_area_x',
 'AVM_Error_Indicators_0_x',
 'AVM_Error_Indicators_6_x',
 'AVM_Error_Indicators_5_x',
 'avmValue_x']

In [10]:
Predictors_ann = noun_sentence_df_final_joined_df_selected_columns_use[predictor_columns_ann].to_numpy()

In [11]:
for i in range(Predictors_ann.shape[1]):
    Predictors_ann[:,i]=(Predictors_ann[:,i]-np.mean(Predictors_ann[:,i]))/np.std(Predictors_ann[:,i])

In [12]:
Predictors = noun_sentence_df_final_joined_df_selected_columns_use_min.loc[:,predictor_columns].to_numpy()

In [13]:
Predictors.shape

(200000, 13)

In [14]:
predictand = noun_sentence_df_final_joined_df_selected_columns_use_min.avmerror.values

In [15]:
predictand.shape

(200000,)

In [16]:
import joblib

modelRF01 = joblib.load('RF_newmodel_01_3mins.sav')
modelXG01 = joblib.load('XG_newmodel_01_3mins.sav')


In [17]:
modelNN01 = keras.models.load_model('RF_newmodel_01_3mins.h5')

In [18]:
predictions_RF01 =  modelRF01.predict(Predictors)
predictions_XG01=  modelXG01.predict(Predictors)
predictions_NN01 =  modelNN01.predict(Predictors_ann)

In [19]:
df = noun_sentence_df_final_joined_df_selected_columns_use.copy()

In [20]:
predictions_RF01_value = df['avmValue'].values/(1-predictions_RF01)
predictions_XG01_value = df['avmValue'].values/(1-predictions_XG01)
predictions_NN01_value = df['avmValue'].values/(1-predictions_NN01[:,0])

In [21]:
predictions_value = (predictions_RF01_value +
                    predictions_NN01_value +
                    predictions_XG01_value )/3.

In [22]:
print('RF01', np.mean(np.abs(df['Transprice'].values-predictions_RF01_value)/\
              (df['Transprice'].values)))
print('XG01',np.mean(np.abs(df['Transprice'].values-predictions_XG01_value)/\
              (df['Transprice'].values)))
print('NN01',np.mean(np.abs(df['Transprice'].values-predictions_NN01_value)/\
              (df['Transprice'].values)))
print('Combined',np.mean(np.abs(df['Transprice'].values-predictions_value)/\
              (df['Transprice'].values)))

print('AVM',np.mean(np.abs(df['Transprice'].values- df['avmValue'].values)/\
              (df['Transprice'].values)))


RF01 0.10509304923462261
XG01 0.10803326476820406
NN01 0.11389768457972728
Combined 0.10825230833459365
AVM 0.12211574741864119
